In [1]:
import os
import json
import traceback

from datetime import datetime

from tqdm.notebook import tqdm

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.config.system_parameters import TriageSettings

from react_agent.src.util.tools_fabric import ToolsFabric

from experiments.models.experiment_models import LabeledQAPairFacts, ExperimentResult
from experiments.metrics.fact_score.fact_scorer import FactScorer
from experiments.metrics.bert_score import BertScore
from experiments.metrics.agent_judge import AgentJudgeEvaluator

In [2]:
USE_MCP = True
DEBUG_MODE = False
USE_TRIAGE = True

AGENT_JUDGE_MODEL = "anthropic--claude-3.5-sonnet"

fact_scorer = FactScorer()
path_to_source_data = "./resources/02_facts/"

date_str = datetime.now().strftime("%d%m%Y")
path_to_results = f"./results/{date_str}"

if not os.path.exists(path_to_results):
    os.makedirs(path_to_results)

time_str = datetime.now().strftime("%H%M%S")

result_file_name = f"{path_to_results}/{time_str}_results.json"
failed_experiments_file_name = f"{path_to_results}/{time_str}_failed_experiments.json"

In [3]:
file_path = os.path.abspath(path_to_source_data + "data_set_facts_copy.json")

with open(file_path, encoding="utf8") as f:
    data = json.load(f)
    data_set = [LabeledQAPairFacts(**item) for item in data]

In [4]:
data_set_for_experiment = data_set

In [5]:
triage_agent = Triage()

In [6]:
async def run_experiment(data_row):
    result = ExperimentResult(
        id=data_row.id,
        question=data_row.question,
        answer=data_row.answer,
        product=data_row.product,
        category=data_row.category,
        persona=data_row.persona,
        activity=data_row.activity,
        country=data_row.country,
    )
    print(f"Asking agent question with id: {data_row.id}")
    if USE_TRIAGE:
        print("\tUsing Triage to categorize the query...")
        query_categorization = triage_agent.triage_user_message(
            user_message=data_row.question
        )
        if DEBUG_MODE:
            print(f"\tQuery categorization: {query_categorization}")

    result.triage_category = (
        query_categorization["category"]
        if USE_TRIAGE
        else TriageSettings.Categories.ALL
    )

    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=result.triage_category,
    )

    if USE_MCP:
        async with MultiServerMCPClient(tools) as client:
            agent = ReActAgent(tool_list=client.get_tools())
            await agent.arun_agent_with_input(
                user_message=data_row.question, debug=DEBUG_MODE
            )
    else:
        agent = ReActAgent(tool_list=tools)
        agent.run_agent_with_input(user_message=data_row.question, debug=DEBUG_MODE)

    run_data = agent.get_execution_data()

    result.tools_used = run_data.tools_used
    result.tool_calls_count = len(run_data.tools_used)
    result.excecution_time_seconds = run_data.excecution_time_seconds
    result.model_used = run_data.model_used
    result.tokens_consumed = run_data.tokens_consumed
    result.llm_call_count = run_data.llm_call_count
    result.facts = data_row.facts
    result.generated_answer = run_data.final_output

    print("\tFinished agent execution")

    print("\tCalculating FactScore...")
    result.fact_score = await fact_scorer.get_fact_score(
        facts=data_row.facts,
        knowledge_source=result.generated_answer,
        debug=DEBUG_MODE,
    )

    print("\tCalculating BERTScore...")
    result.bert_score = BertScore.compute_score(
        expected_response=data_row.answer, actual_response=result.generated_answer
    )

    print("\tEvaluating agent response with Agent as a Judge...")
    result.agent_judge_model = AGENT_JUDGE_MODEL
    agent_evaluator = AgentJudgeEvaluator(model=AGENT_JUDGE_MODEL)

    agent_judge_outcome = agent_evaluator.evaluate(
        question=data_row.question,
        expert_answer=data_row.answer,
        generated_answer=result.generated_answer,
    )

    result.agent_judge_outcome = agent_judge_outcome.answer
    result.agent_judge_reasoning = agent_judge_outcome.reasoning

    result.agent_judge_call_count = agent_evaluator.get_llm_call_count()
    result.agent_judge_tokens_consumed = agent_evaluator.get_token_consumption()

    print(f"\tFinished experiment for id: {data_row.id}\n")

    return result

In [7]:
experiment_results: list[ExperimentResult] = []
experiment_to_retry: list = []

for data_row in tqdm(data_set_for_experiment, desc="Running cases", unit="case"):
    try:
        result = await run_experiment(data_row)

        experiment_results.append(result)
    except Exception as e:
        print(f"\tRetrying experiment for id {data_row.id}: {e}")

        experiment_to_retry.append(data_row)

Running cases:   0%|          | 0/1 [00:00<?, ?case/s]

Asking agent question with id: PS-3
	Using Triage to categorize the query...
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with Agent as a Judge...
	Finished experiment for id: PS-3



In [8]:
failed_experiments = []

for data_row in tqdm(experiment_to_retry, desc="Running cases", unit="case"):
    try:
        result = await run_experiment(data_row)

        experiment_results.append(result)
    except Exception as e:
        print(f"\tExperiment failed for id {data_row.id}: {e}")
        if DEBUG_MODE:
            traceback.print_exc()

        failed_experiments.append(
            {
                "id": data_row.id,
                "error": str(e),
                "traceback": traceback.format_exc(),
            }
        )

Running cases: 0case [00:00, ?case/s]

In [9]:
if failed_experiments:
    with open(failed_experiments_file_name, "w") as f:
        json.dump(failed_experiments, f, indent=2)

In [10]:
# Serialize experiment results to list of dicts
import pandas as pd

records = [r.model_dump(mode="json") for r in experiment_results]
# Make it a dataframe in order to flatten nested structures
df = pd.json_normalize(records)

df

/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(direct_fa...oing E-Mail Process.")])` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'Knowledge-QA'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


,id,question,answer,product,category,persona,activity,country,facts,bert_score,...,fact_score.supporting_fact_score,fact_score.combined_fact_score,fact_score.direct_facts,fact_score.supporting_facts,agent_judge_tokens_consumed.input_tokens,agent_judge_tokens_consumed.output_tokens,agent_judge_tokens_consumed.total_tokens,tokens_consumed.input_tokens,tokens_consumed.output_tokens,tokens_consumed.total_tokens
0,PS-3,Incoming ZUGFeRD invoices (from suppliers) we ...,Here are some hints that might be helpful. We ...,ERP (excluding SAP S/4HANA Cloud Public Edition),How to Develop (Maintenance - Customer case su...,Developer (Customer side),Custom Development,DE,[{'fact': 'You can extend EDOPROCFUNCASGV as i...,0.057506,...,0.0,0.0,[{'fact': 'You can extend EDOPROCFUNCASGV as i...,[{'fact': 'There are screenshots available sho...,4748,3,4751,14890,2009,16899


In [11]:
# Convert to list of dictionaries (JSON objects)
json_list = df.to_dict(orient="records")

with open(result_file_name, "w") as f:
    json.dump(json_list, f, indent=2)